<a href="https://colab.research.google.com/github/Vaishnavi-Chandrashekar/CC-FRAUD-DETECTION/blob/main/Data_Preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dataset link
https://www.kaggle.com/competitions/nus-fintech-recruitment/overview/description

# Dataset Description
The objective of this competition is to predict whether a transaction is fraudulent or legitimate. The target label is TX_FRAUD, where 1 indicates a fraudulent transaction and 0 indicates a legitimate transaction.

Please take note that the test.csv file contains credit card transactions that occurred from Jan - Apr 2022. However, for the final submission of this competition, please only submit the predictions for transactions that occurred in Apr 2022 and sort them by transaction ID in ascending order (from transaction_id = 524230 at the top to transaction_id = 579320 at the bottom). Please refer to the sample_submission.csv file for your final submission.

# Files
* train.csv - the training set. Contains credit card transactions that occurred from Aug - Dec 2021
* test.csv - the test set for submission. Contains credit card transactions that occurred from Jan - Apr 2022. Only predictions for transactions that occurred in Apr 2022 need to be submitted. Please sort your submission by the transaction ID in ascending order
* sample_submission.csv - a sample submission file in the correct format
* customer.csv - list of customers and their respective customer IDs. A pair of coordinates (x_customer_id, y_customer_id) is provided to indicate the location of the customer
* terminal.csv - list of merchants (terminals) and their respective terminal IDs. A pair of coordinates (x_terminal_id, y_terminal_id) is provided to indicate the location of the terminal

# Columns in train.csv
* TRANSACTION_ID - transaction ID
* TX_DATETIME - date and time of transaction
* CUSTOMER_ID - customer ID involved in the transaction
* TERMINAL_ID - terminal ID where transaction occurred
* TX_AMOUNT - amount transacted
* TX_FRAUD - indicates if the transaction is fraudulent. 1 for fraudulent and 0 for legitimate

# Columns in customer.csv
* CUSTOMER_ID - customer ID
* x_customer_id - x-coordinate of the customer
* y_customer_id - y-coordinate of the customer
* mean_amount - mean amount spent by the customer
* std_amount - standard deviation of the amount spent by the customer
* mean_nb_tx_per_day - mean number of transactions made by the customer per day
* available_terminals - terminals where the customer is able to make transactions. We assume that customers can only make transactions at terminals within a radius of 5 units from the location of the customer
* nb_terminals - number of terminals that the customer can make transactions

# Columns in terminal.csv
* TERMINAL_ID - terminal ID
* x_terminal_id - x-coordinate of the terminal
* y_terminal_id - y-coordinate of the terminal

In [79]:
#importing the common libraries
import numpy as np
import pandas as pd

# Train data

In [80]:
#importing the train data
train_data = pd.read_csv('train.csv')

In [81]:
#reading the train data
train_data.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_FRAUD
0,59383,2021-08-01 00:04:37,323,217,4.60,0
1,59384,2021-08-01 00:12:10,6,429,8.61,0
2,59385,2021-08-01 00:12:34,714,1011,64.00,0
3,59386,2021-08-01 00:15:40,266,1969,12.72,0
4,59387,2021-08-01 00:16:01,890,1482,98.88,0


In [82]:
train_data.shape

(291231, 6)

In [83]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 291231 entries, 0 to 291230
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   TRANSACTION_ID  291231 non-null  int64  
 1   TX_DATETIME     291231 non-null  object 
 2   CUSTOMER_ID     291231 non-null  int64  
 3   TERMINAL_ID     291231 non-null  int64  
 4   TX_AMOUNT       291231 non-null  float64
 5   TX_FRAUD        291231 non-null  int64  
dtypes: float64(1), int64(4), object(1)
memory usage: 13.3+ MB


In [84]:
train_data.isnull().sum()

TRANSACTION_ID    0
TX_DATETIME       0
CUSTOMER_ID       0
TERMINAL_ID       0
TX_AMOUNT         0
TX_FRAUD          0
dtype: int64

In [85]:
train_data.duplicated().sum()

0

In [86]:
train_data.describe()

,TRANSACTION_ID,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_FRAUD
count,291231.000000,291231.000000,291231.000000,291231.000000,291231.000000
mean,204998.000000,496.256202,993.718526,53.182274,0.022601
std,84071.292461,285.274966,575.267551,39.573329,0.148627
min,59383.000000,0.000000,0.000000,0.000000,0.000000
25%,132190.500000,249.000000,496.000000,21.130000,0.000000
50%,204998.000000,498.000000,1000.000000,44.770000,0.000000
75%,277805.500000,741.000000,1483.000000,76.850000,0.000000
max,350613.000000,999.000000,1999.000000,291.150000,1.000000


Observation:
* There are no null values in the train data.
* There are no duplicates in the train data.
* All the columns except the data time column are numerical.

#Customer data

In [87]:
#importing the customer data
customer_data = pd.read_csv('customer.csv')

In [88]:
#reading the customer data
customer_data.head()

,CUSTOMER_ID,x_customer_id,y_customer_id,mean_amount,std_amount,mean_nb_tx_per_day,available_terminals,nb_terminals
0,0,54.881350,71.518937,62.262521,31.131260,2.179533,"[29, 87, 144, 241, 330, 858, 996, 1028, 1067, ...",22
1,1,42.365480,64.589411,46.570785,23.285393,3.567092,"[5, 160, 242, 378, 431, 475, 571, 762, 876, 93...",20
2,2,96.366276,38.344152,80.213879,40.106939,2.115580,"[316, 406, 447, 523, 968, 1200, 1318, 1365, 16...",10
3,3,56.804456,92.559664,11.748426,5.874213,0.348517,"[65, 94, 113, 364, 401, 433, 485, 651, 672, 77...",17
4,4,2.021840,83.261985,78.924891,39.462446,3.480049,"[372, 614, 774, 1362, 1446, 1564, 1637, 1939]",8


In [89]:
customer_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   CUSTOMER_ID          1000 non-null   int64  
 1   x_customer_id        1000 non-null   float64
 2   y_customer_id        1000 non-null   float64
 3   mean_amount          1000 non-null   float64
 4   std_amount           1000 non-null   float64
 5   mean_nb_tx_per_day   1000 non-null   float64
 6   available_terminals  1000 non-null   object 
 7   nb_terminals         1000 non-null   int64  
dtypes: float64(5), int64(2), object(1)
memory usage: 62.6+ KB


In [90]:
customer_data.shape

(1000, 8)

In [91]:
customer_data.isnull().sum()

CUSTOMER_ID            0
x_customer_id          0
y_customer_id          0
mean_amount            0
std_amount             0
mean_nb_tx_per_day     0
available_terminals    0
nb_terminals           0
dtype: int64

In [92]:
customer_data.duplicated().sum()

0

Observation:
* There are no null values in the customer data.
* There are no duplicates in the customer data.
* All the columns except the available_terminals column are numerical.

#Terminal data

In [93]:
#importing the terminal data
terminal_data = pd.read_csv('terminal.csv')

In [94]:
#reading the terminal data
terminal_data.head()

,TERMINAL_ID,x_terminal_id,y_terminal_id
0,0,41.702200,72.032449
1,1,0.011437,30.233257
2,2,14.675589,9.233859
3,3,18.626021,34.556073
4,4,39.676747,53.881673


In [95]:
terminal_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   TERMINAL_ID    2000 non-null   int64  
 1   x_terminal_id  2000 non-null   float64
 2   y_terminal_id  2000 non-null   float64
dtypes: float64(2), int64(1)
memory usage: 47.0 KB


In [96]:
terminal_data.shape

(2000, 3)

In [97]:
terminal_data.isnull().sum()

TERMINAL_ID      0
x_terminal_id    0
y_terminal_id    0
dtype: int64

In [98]:
terminal_data.duplicated().sum()

0

Observation:
* There are no null values in the train data.
* There are no duplicates in the train data.
* All the columns are numerical.

# Merging the customer data to train data

In [99]:
train_df = pd.merge(train_data,customer_data, on='CUSTOMER_ID',how='left')

# Merging the terminal data to train data

In [100]:
train_df = pd.merge(train_df,terminal_data, on='TERMINAL_ID', how='left')

In [101]:
train_df.shape

(291231, 15)

In [102]:
train_df.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_FRAUD,x_customer_id,y_customer_id,mean_amount,std_amount,mean_nb_tx_per_day,available_terminals,nb_terminals,x_terminal_id,y_terminal_id
0,59383,2021-08-01 00:04:37,323,217,4.60,0,84.515409,38.276422,7.353061,3.676530,3.324124,"[51, 68, 208, 217, 293, 353, 534, 717, 773, 86...",19,80.316334,40.239252
1,59384,2021-08-01 00:12:10,6,429,8.61,0,11.827443,63.992102,18.618562,9.309281,3.778676,"[163, 172, 205, 429, 468, 607, 750, 786, 881, ...",16,15.172487,63.912685
2,59385,2021-08-01 00:12:34,714,1011,64.00,0,75.221083,94.991427,82.620413,41.310207,3.723765,"[58, 799, 1011, 1021, 1228, 1347, 1443, 1462, ...",13,74.196424,98.288079
3,59386,2021-08-01 00:15:40,266,1969,12.72,0,51.122179,8.329098,9.852171,4.926085,3.862067,"[27, 493, 584, 734, 820, 917, 1108, 1363, 1444...",15,51.950635,6.563484
4,59387,2021-08-01 00:16:01,890,1482,98.88,0,62.777887,31.405270,83.660035,41.830018,3.128315,"[154, 177, 351, 444, 633, 739, 1018, 1056, 113...",20,62.417972,26.483666


In [103]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 291231 entries, 0 to 291230
Data columns (total 15 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   TRANSACTION_ID       291231 non-null  int64  
 1   TX_DATETIME          291231 non-null  object 
 2   CUSTOMER_ID          291231 non-null  int64  
 3   TERMINAL_ID          291231 non-null  int64  
 4   TX_AMOUNT            291231 non-null  float64
 5   TX_FRAUD             291231 non-null  int64  
 6   x_customer_id        291231 non-null  float64
 7   y_customer_id        291231 non-null  float64
 8   mean_amount          291231 non-null  float64
 9   std_amount           291231 non-null  float64
 10  mean_nb_tx_per_day   291231 non-null  float64
 11  available_terminals  291231 non-null  object 
 12  nb_terminals         291231 non-null  int64  
 13  x_terminal_id        291231 non-null  float64
 14  y_terminal_id        291231 non-null  float64
dtypes: float64(8), in

In [104]:
train_df.duplicated().sum()

0

In [109]:
train_df.to_csv('train_df.csv', index=False)